In [ ]:

## to do: 
# smiles and FF matching --> topologie für die atom types
# lammps building template


In [16]:

# smiles = [ "CCCC", "CCCCCC" ]
# force_field_paths = [ "/Users/samir/Documents/Coding_libaries/pyLMP/force-fields/forcefield_UA_TAMie_alkanes.toml" ]

# ff = forcefield( smiles = smiles, force_field_paths = force_field_paths )

# residues = ["butane","hexane"]


# files_suffix = "mol"

# template = "/Users/samir/Documents/Coding_libaries/pyLMP/templates/lammps/template_lammps.mol"

# #ff.write_molecule_files( template, ".", residues, files_suffix, **kwargs )


Force field provided for software 'LAMMPS'
